In [22]:
# This is a project type notebook, it needs huge computational power and time for the full IMDB Dataset

In [23]:
!pip install nltk

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saims\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd

In [4]:
# importing the dataframe

In [5]:
# df = pd.read_csv('IMDB Dataset.csv')

# taking only first 5000 rows
df = pd.read_csv('IMDB Dataset.csv').head(100)

In [6]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
95,Daniel Day-Lewis is the most versatile actor a...,positive
96,My guess would be this was originally going to...,negative
97,"Well, I like to watch bad horror B-Movies, cau...",negative
98,"This IS the worst movie I have ever seen, as w...",negative


In [7]:
# EDA

In [8]:
df.shape

(100, 2)

In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
df.describe()

,review,sentiment
count,100,100
unique,100,2
top,One of the other reviewers has mentioned that ...,negative
freq,1,58


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     100 non-null    object
 1   sentiment  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [12]:
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [13]:
df['sentiment'].value_counts()

negative    58
positive    42
Name: sentiment, dtype: int64

In [14]:
# apply label encoder

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [16]:
df['sentiment'] = le.fit_transform(df['sentiment'])

In [17]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
X = df['review']
y = df['sentiment']

In [19]:
X

0     One of the other reviewers has mentioned that ...
1     A wonderful little production. <br /><br />The...
2     I thought this was a wonderful way to spend ti...
3     Basically there's a family where a little boy ...
4     Petter Mattei's "Love in the Time of Money" is...
                            ...                        
95    Daniel Day-Lewis is the most versatile actor a...
96    My guess would be this was originally going to...
97    Well, I like to watch bad horror B-Movies, cau...
98    This IS the worst movie I have ever seen, as w...
99    I have been a Mario fan for as long as I can r...
Name: review, Length: 100, dtype: object

In [20]:
y

0     1
1     1
2     1
3     0
4     1
     ..
95    1
96    0
97    0
98    0
99    1
Name: sentiment, Length: 100, dtype: int32

In [21]:
# removing all special and numeric character from the data and also remove stopwords and apply stemming

In [22]:
# for stlyling only
from tqdm import tqdm

In [23]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

In [24]:
ps = PorterStemmer()
corpus = []

for i in tqdm(range(len(X))):
#     print(i, end=', ')
    review = re.sub("[^a-zA-Z]"," ",X[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 31.30it/s]


In [28]:
print(corpus)

['one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch d

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
# cv = TfidfVectorizer(max_features=5000)
cv = TfidfVectorizer(max_features=500)
X = cv.fit_transform(corpus).toarray()

In [27]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.11142743, 0.        ,
        0.12091984],
       ...,
       [0.        , 0.08655773, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [65]:
X.shape

(5000, 500)

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=81)

In [67]:
from sklearn.svm import SVC
model = SVC(C=1.0,
           kernel='linear',
           random_state=81)

In [68]:
model.fit(X_train, y_train)

SVC(kernel='linear', random_state=81)

In [69]:
model.score(X_test, y_test)

0.854

In [70]:
y_pred = model.predict(X_test)

In [71]:
# save and load the model
# import pickle

In [72]:
# pickle.dump(cv, open('movie_review_cv.pkl', 'wb'))
# pickle.dump(model, open('movie_review_model.pkl', 'wb'))

In [73]:
# save_cv = pickle.load(open('movie_review_cv.pkl', 'rb'))
# save_model = pickle.load( open('movie_review_model.pkl', 'rb'))

In [78]:
# function for testing

def test_model(sentence):
#     sen = save_cv.transform([sentence]).toarray()
    sen = cv.transform([sentence]).toarray()
#     res = save_model.predict(sen)[0]
    res = model.predict(sen)[0]
    #     print(res)
    if res == 1:
        print('positive')
    else:
        print('negative')

In [79]:
sentence = 'i love amber heard'
test_model(sentence)

positive


In [80]:
sentence = 'i hate gigi hadid'
test_model(sentence)

negative


In [81]:
# print(set(stopwords.words("english")))

In [82]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86       507
           1       0.85      0.85      0.85       493

    accuracy                           0.85      1000
   macro avg       0.85      0.85      0.85      1000
weighted avg       0.85      0.85      0.85      1000

